## Importing Libraries

In [ ]:
import PIL.Image as Image
import cv2
import numpy as np

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers,losses
from tensorflow.keras.models import Sequential

### Unziping files

In [ ]:
import os
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/archive.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

### Making Path Object

In [ ]:
import pathlib                               
train_dir = pathlib.Path("/content/dataset/training_set")
test_dir=pathlib.Path("/content/dataset/test_set")

print(train_dir)
print(test_dir)

In [ ]:
list(train_dir.glob('*/*'))[:5] 

In [ ]:
cat_train= list(train_dir.glob('cats/*'))
len(cat_train)

In [ ]:
image_count= len(list(train_dir.glob('*/*.jpg')))
image_count

In [ ]:
Image.open(str(cat_train[1500]))

In [ ]:
dog_train= list(train_dir.glob('dogs/*'))
len(cat_train)

In [ ]:
Image.open(str(dog_train[1569]))

## Creating Training Dataset

In [ ]:
images_dict = {
    'cats': cat_train[:2200],    
    'dogs': dog_train[:2200],
}

In [ ]:
images_dict["dogs"][:5]

In [ ]:
labels_dict = {
    'cats': 0,
    'dogs': 1,
    
}

In [ ]:
img= cv2.imread(str(images_dict['cats'][0]))

In [ ]:
img.shape

In [ ]:

X, y = [], []
for name,images in images_dict.items():#returns key , path for each record
    for image in images:
        img=cv2.imread(str(image))
     
        resize_img=cv2.resize(img,(224,224))
       
        X.append(resize_img)
        y.append(labels_dict[name])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape,y.shape

## Splitting in Train - Validation

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30 ,random_state=0)

## Model building

In [ ]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [ ]:
model = Sequential([
                pretrained_model_without_top_layer,
                tf.keras.layers.Dense(2)
])

In [ ]:
model.summary()

In [ ]:
# Scaling the image 
X_train_scaled = X_train / 255
X_val_scaled = X_val / 255

##Model training

In [ ]:
model.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc']
)
history = model.fit(X_train_scaled, y_train,validation_data=(X_val_scaled,y_val),epochs=5)

##Plotting Accuracy and Loss

In [ ]:
plt.figure(figsize=(15,5))
epochs=range(1,len(history.history['val_acc'])+1)

plt.plot(epochs,history.history['acc'])
plt.plot(epochs,history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
epochs=range(1,len(history.history['val_loss'])+1)

plt.plot(epochs,history.history['loss'])
plt.plot(epochs,history.history['val_loss'])
plt.title('model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

## Making Test dataset for Prediction

In [ ]:
cat_test= list(test_dir.glob('cats/*'))
dog_test= list(test_dir.glob('dogs/*'))

In [ ]:
len (dog_test)

In [ ]:
test_images_dict = {
    'cats': cat_test,    
    'dogs': dog_test,
}

test_labels_dict = {
    'cats': 0,
    'dogs': 1,
    
}

In [ ]:
X_test, y_test = [], []
for name,images in test_images_dict.items():#returns key , path for each record
    for image in images:
        img=cv2.imread(str(image))
     
        resize_img=cv2.resize(img,(224,224))
       
        X_test.append(resize_img)
        y_test.append(test_labels_dict[name])

In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
X_test.shape

In [ ]:
# Scaling the image 
X_test_scaled = X_test / 255

## Fitting test data to check accuracy

In [ ]:
model.evaluate(X_test_scaled,y_test)